# DIN

只测试交互模块，没有实现完整的训练过程


In [1]:
%cd /playground/sgd_deep_learning/sgd_rec_sys/
import sys
sys.path.append('./python')

/playground/sgd_deep_learning/sgd_rec_sys


In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import torch.optim as optim

import pickle
import numpy as np

from sgd_rec_sys.lastn import DIN
from sgd_rec_sys.data import FakeLastnDataFactory, LastnDataset

## lastn fake data生成

In [3]:
# input
n_samples = 600 # 总样本数
last_n = 10 # 与用户最后10个物品做交互
fea_dim = 32 # 用户单个行为concat后的编码维度

in_dim = int(fea_dim*(fea_dim+2)) # activation Unit中，外积扩张后的维度
hidden_dim = int(max(16, in_dim/2)) # 隐藏层

## 生成伪Lastn数据
dump_file = './data/fake/lastn.pkl'
fake_data_factory = FakeLastnDataFactory(n_samples, fea_dim,
                                        last_n, dtype=np.float32)
fake_data_factory.presist(dump_file)

user_behavior success, shape: (600, 10, 32)
candidate_ad success, shape: (600, 32)


## 参数设置

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('device:', device)

# 训练参数
train_batch_size = 64
epochs = 1

with open(dump_file,'rb') as f:
    fake_data = pickle.load(f)
train_ds = LastnDataset(fake_data, device)
train_dl = DataLoader(train_ds, batch_size=train_batch_size, shuffle=True)


# for  x in train_dl:
#     a,b = x
#     print(a.shape, b.shape, a.device)
# 输入验证 torch.Size([64, 32]) torch.Size([64, 10, 32]) cuda:0

# 定义模型
model = DIN(fea_dim,
            hidden_dim,
            ).to(device)


device: cuda


## Forward

In [5]:
def train(dataloader, model, epochs=1,):
    for epoch in range(epochs):
        for x in dataloader:
            # candidate_ad, goods = x # [B,K] [B,N,K]
            outputs = model(x) # forward
            print('sum pooling out:', outputs.shape)
            
            print('B, N, K', list(x[1].shape))
            
            break
            
train(train_dl, model, epochs)

sum pooling out: torch.Size([64, 32])
B, N, K [64, 10, 32]
